# 图像金字塔与图片缩放

#1 概述
我们经常会将某种尺寸的图像转换为其他尺寸的图像，如果放大或者缩小图片的尺寸，笼统来说的话，可以使用opencv为我们提供的如下两种方式：
1. resize函数。这是最直接的方式，
2. pyrUp()、pyrDown()函数：即图像金字塔相关的两个函数，对图像进行向上采样，向下采样的操作。pyrUp、pyrDown其实和专门用作放大缩小图像尺寸的resize在功能上差不多，披着图像金字塔的皮，说白了还是在对图像进行放大和缩小操作。

#2 图像金字塔
图像金字塔是图像中多尺度表达的一种，最主要用于图像的分割，是一种以多分辨率来解释图像的有效但概念简单的结构。图像金字塔最初用于机器视觉和图像压缩，一幅图像的金字塔是一系列以金字塔形状排列的分辨率逐步降低，且来源于同一张原始图的图像集合。其通过梯次向下采样获得，直到达到某个终止条件才停止采样。金字塔的底部是待处理图像的高分辨率表示，而顶部是低分辨率的近似。我们将一层一层的图像比喻成金字塔，层级越高，则图像越小，分辨率越低。![images](../images/12/24.png)主要有两种类型的金字塔：高斯金字塔和拉普拉斯金字塔

当图像向金字塔的上层移动时，尺寸和分辨率就降低。OpenCV中，从金字塔中上一级图像生成下一级图像的可以用PryDown。而通过PryUp将现有的图像在每个维度都放大两遍。图像金字塔中的向上和向下采样分别通过OpenCV函数pyrUp和pyrDown实现。
概括起来就是：
- 对图像向上采样：pyrUp函数
- 对图像向下采样：pyrDown函数

这里的向下与向上采样，是对图像的尺寸而言的（和金字塔的方向相反），向上就是图像尺寸加倍，向下就是图像尺寸减半。而如果我们按上图中演示的金字塔方向来理解，金字塔向上图像其实在缩小，这样刚好是反过来了。
 
但需要注意的是，PryUp和PryDown不是互逆的，即PryUp不是降采样的逆操作。这种情况下，图像首先在每个维度上扩大为原来的两倍，新增的行（偶数行）以0填充。然后给指定的滤波器进行卷积（实际上是一个在每个维度都扩大为原来两倍的过滤器）去估计“丢失”像素的近似值。
PryDown( )是一个会丢失信息的函数。为了恢复原来更高的分辨率的图像，我们要获得由降采样操作丢失的信息，这些数据就和拉普拉斯金字塔有关系了。

##2.1 高斯金字塔
用来向下采样，主要的图像金字塔。高斯金字塔是通过高斯平滑和下采样获得一些列下采样图像，也就是说第K层高斯金字塔通过平滑、下采样就可以获得K+1层高斯图像，高斯金字塔包含了一系列低通滤波器，其截至频率从上一层到下一层是以因子2逐渐增加，所以高斯金字塔可以跨越很大的频率范围。![images](../images/12/18.png)

高斯核是唯一可以产生多尺度空间的核，因此用高斯平滑得到的金字塔图像通常也叫做高斯金字塔.高斯核是实现尺度变换的唯一线性核。并且满足平移不变性、半群结构、非增局部极值、尺度不变性和旋转不变形等性质

###2.1.1 图像下采样
为了获取层级为$G_{i+1}$的金字塔图像，我们采用如下方法:
- 对图像$G_i$进行高斯内核卷积
- 将所有偶数行和列去除

得到的图像即为$G_{i+1}$的图像，显而易见，结果图像只有原图的四分之一。通过对输入图像$G_i$(原始图像)不停迭代以上步骤就会得到整个金字塔。同时我们也可以看到，向下取样会逐渐丢失图像的信息。以上就是对图像的向下取样操作，即缩小图像。

###2.1.2 对图像的向上取样
如果想放大图像，则需要通过向上取样操作得到，具体做法如下：
- 将图像在每个方向扩大为原来的两倍，新增的行和列以0填充
- 使用先前同样的内核(乘以4)与放大后的图像卷积，获得 “新增像素”的近似值

得到的图像即为放大后的图像，但是与原来的图像相比会发觉比较模糊，因为在缩放的过程中已经丢失了一些信息，如果想在缩小和放大整个过程中减少信息的丢失，这些数据形成了拉普拉斯金字塔。

##2.2 拉普拉斯金字塔(Laplacian)
用来从金字塔低层图像重建上层未采样图像，在数字图像处理中也即是预测残差，可以对图像进行最大程度的还原，配合高斯金字塔一起使用.高频细节信息在卷积核和降采样中丢失，拉普拉斯金字塔可以保留所有层所丢失的高频信息，用于图像恢复$L_i=G_i-UP(G_{i+1}) \oplus \psi_{5*5}$，其中$G_i$标识第i层的图像，而$UP()$操作是讲源图像中位置为$(x,y)$的像素映射到目标图像的$(2x+1,2y+1)$的位置，即在进行上采样，$\oplus$表示卷积,$\psi_{5*5}$为$5*5$的高斯内核。第i层的拉普拉斯就等于第i层的高斯减去对第i+1层的高斯结果进行上采样乘以一个高斯卷积核.因此我们可以直接用OpenCV进行拉普拉斯运算：$L_i=G_i-PyrUp(G_{i+1})$。拉普拉斯金字塔是通过源图像减去先缩小后再放大的图像的一系列图像构成的

由公式可以看出，拉普拉斯金字塔需要跟高斯金字塔配合才能进行图像恢复；如果图像不会被恢复，那么就没有必要做拉普拉斯金字塔
![images](../images/12/19.png)
红框中是做的高斯金字塔操作，绿框做的是拉普拉斯金字塔操作，蓝框做的是图片的恢复操作。其中，红框和蓝框中的三张图片是一样的

所以，我们可以将拉普拉斯金字塔理解为高斯金字塔的逆形式。
 
另外再提一点，关于图像金字塔非常重要的一个应用就是实现图像分割。图像分割的话，先要建立一个图像金字塔，然后在G_i和G_i+1的像素直接依照对应的关系，建立起”父与子“关系。而快速初始分割可以先在金字塔高层的低分辨率图像上完成，然后逐层对分割加以优化。


##2.3 区别
两者的简要区别：高斯金字塔用来向下降采样图像，而拉普拉斯金字塔则用来从金字塔底层图像中向上采样重建一个图像。
要从金字塔第i层生成第i+1层（我们表示第i+1层为G_i+1），我们先要用高斯核对G_1进行卷积，然后删除所有偶数行和偶数列。当然的是，新得到图像面积会变为源图像的四分之一。按上述过程对输入图像G_0执行操作就可产生出整个金字塔。

#3 resize函数
关于resize函数，主要注意它的插值方式就好

可选的插值方式如下：
- INTER_NEAREST：最近邻插值
- INTER_LINEAR：线性插值（默认值）
- INTER_AREA：区域插值（利用像素区域关系的重采样插值）
- INTER_CUBIC：三次样条插值（超过4×4像素邻域内的双三次插值）
- INTER_LANCZOS4：Lanczos插值（超过8×8像素邻域的Lanczos插值）

若要缩小图像，一般情况下最好用CV_INTER_AREA来插值，而若要放大图像，一般情况下最好用CV_INTER_CUBIC（效率不高，慢，不推荐使用）或CV_INTER_LINEAR（效率较高，速度较快，推荐使用）。

#4 尺度空间与金字塔空间
尺度空间表达和金字塔多分辨率表达之间最大的不同是：
- 尺度空间表达是由不同高斯核平滑卷积得到，在所有尺度上有相同的分辨率；
- 金字塔多分辨率表达每层分辨率减少固定比率。

所以，金字塔多分辨率生成较快，且占用存储空间少；而多尺度表达随着尺度参数的增加冗余信息也变多。多尺度表达的优点在于图像的局部特征可以用简单的形式在不同尺度上描述；而金字塔表达没有理论基础，难以分析图像局部特征。